In [55]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.7 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/BFB//data.csv')

In [4]:
df.head()

,query,product_id,product_title,product_brand,product_color,product,esci
0,colelge jewelry,B07QY2XZ2D,product_title mDesign Plastic Stacking Jewelry...,product_brand mDesign,product_color Clear,product_title mDesign Plastic Stacking Jewelry...,C
1,mr clean,B06XT161H7,product_title Mr. Clean Multipurpose Cleaning ...,product_brand Mr.Clean,product_color Green,product_title Mr. Clean Multipurpose Cleaning ...,E
2,cotton trousers men,B07YY1QMCN,product_title Wohthops Sport Trousers for Men ...,product_brand Wohthops,product_color 2# Navy,product_title Wohthops Sport Trousers for Men ...,E
3,michael corse wallet for women,B07PPTTRD4,product_title Michael Kors Jet Set Travel Flat...,product_brand Michael Kors,product_color Black,product_title Michael Kors Jet Set Travel Flat...,E
4,womens tank tops,B079SRS8GB,"product_title Next Level Womens Festival Tank,...",product_brand Next Level,product_color Royal Pine,"product_title Next Level Womens Festival Tank,...",E


In [22]:
product_ids = df['product_id']

In [44]:
X = df[['product_id']]
X['text'] = df['query']+['product']

<ipython-input-44-7392fdc604d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['text'] = df['query']+['product']


In [45]:
X.sample(10)

,product_id,text
1935,1982141468,mary trump too much and never enoughproduct
6008,B00DSICMGG,takumi 86product
3447,B07NX2VK6K,printable sticker sheetsproduct
357,B06Y1TPPN1,white fold dianobd earribgs screw bacjproduct
6225,B00NAK8VAS,always discreet underwear smallproduct
5401,B00HF7W2JS,korean cheese ballproduct
3333,B00EPPVWGC,buchiproduct
4490,B07PWJQMNZ,erodeproduct
9382,B00L5YLVGM,add two cups brassiere for women push up padde...
8219,B07C86S13K,hp laptops 17 inch screenproduct


In [16]:
df['esci'].unique()

array(['C', 'E', 'I', 'S'], dtype=object)

In [17]:
mapValues = {'C':0,'E':1,'I':2,'S':3}

In [18]:
y = df['esci'].map(mapValues)

In [21]:
y.sample(10)

939     1
6729    1
4707    3
4538    1
5047    3
3672    2
4401    1
1182    0
5126    1
6749    1
Name: esci, dtype: int64

In [56]:
from sentence_transformers import SentenceTransformer

In [90]:
model_vector = SentenceTransformer('all-mpnet-base-v2')

In [58]:
X["vector"] = X["text"].apply(lambda x: model_vector.encode(x))

<ipython-input-58-ccf499561f0b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["vector"] = X["text"].apply(lambda x: model.encode(x))


In [79]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
from sklearn.preprocessing import StandardScaler

In [60]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [47]:
product_ids_test = X_test['product_id']

In [61]:
X_train.drop(['product_id','text'],axis=1,inplace=True)
X_test.drop(['product_id','text'],axis=1,inplace=True)

In [68]:
X_train_1 = x = np.array(list(map(np.int_,X_train['vector'])))
X_test_1 = x = np.array(list(map(np.int_,X_test['vector'])))

In [80]:
scaler= StandardScaler()
scaler.fit_transform(X_train_1)
scaler.fit(X_test_1)

StandardScaler()

In [81]:
X_train_1.shape

(8000, 768)

In [82]:
# Train a Support Vector Machine (SVM) classifier
model = SVC(kernel='rbf')
model.fit(X_train_1, y_train)

SVC()

In [83]:
# Evaluate the model on the testing set
y_pred = model.predict(X_test_1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.69      1.00      0.82      1389
           2       0.00      0.00      0.00       172
           3       0.00      0.00      0.00       400

    accuracy                           0.69      2000
   macro avg       0.17      0.25      0.20      2000
weighted avg       0.48      0.69      0.57      2000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   0,   39,    0,    0],
       [   0, 1389,    0,    0],
       [   0,  172,    0,    0],
       [   0,  400,    0,    0]])

#Encode just product title and query


In [88]:
X_title = df[['product_id']]
X_title['text'] = df['query']+['product_title']

<ipython-input-88-02d568adabdd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_title['text'] = df['query']+['product_title']


In [91]:
X_title["vector"] = X_title["text"].apply(lambda x: model_vector.encode(x))

<ipython-input-91-c5a8200bf90b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_title["vector"] = X_title["text"].apply(lambda x: model_vector.encode(x))


In [103]:
# Split the dataset into training and testing sets
X_title_train, X_title_test, y_title_train, y_title_test = train_test_split(X_title, y, test_size=0.1, random_state=42)

In [104]:
product_ids_title_test = X_title_test['product_id']

In [105]:
X_title_train.drop(['product_id','text'],axis=1,inplace=True)
X_title_test.drop(['product_id','text'],axis=1,inplace=True)

In [106]:
X_title_train_1 = x = np.array(list(map(np.int_,X_title_train['vector'])))
X_title_test_1 = x = np.array(list(map(np.int_,X_title_test['vector'])))

In [107]:
scaler= StandardScaler()
scaler.fit_transform(X_title_train_1)
scaler.fit(X_title_test_1)

StandardScaler()

In [108]:
# Train a Support Vector Machine (SVM) classifier
model = SVC(kernel='rbf')
model.fit(X_title_train_1, y_title_train)

SVC()

In [109]:
# Evaluate the model on the testing set
y_title_pred = model.predict(X_title_test_1)
print(classification_report(y_title_test, y_title_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.69      1.00      0.82       691
           2       0.00      0.00      0.00        83
           3       0.00      0.00      0.00       210

    accuracy                           0.69      1000
   macro avg       0.17      0.25      0.20      1000
weighted avg       0.48      0.69      0.56      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [110]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_title_test, y_title_pred)

array([[  0,  16,   0,   0],
       [  0, 691,   0,   0],
       [  0,  83,   0,   0],
       [  0, 210,   0,   0]])